# Identificação de *Motifs*

In [6]:
import docker, os

a = %pwd
wd = (a.rsplit('/',2))[0]

GENE = "ORF7a"
BLAST_Results_ncbi=os.path.join("homologue", GENE, "MN908947.3_ORF7a.fasta")
BLAST_Results_uniprot=os.path.join("homologue", GENE, "B20210130A94466D2655679D1FD8953E075198DA8030B5FR.fasta")
REF=os.path.join("reference", GENE, "P0DTC7.fasta")
D_MEME='quay.io/biocontainers/meme:5.3.0--py37pl526h072abfd_0'

WORK_HOST_DIRECTORY=wd
WORK_CONT_DIRECTORY='/opt/project'

In [7]:
from Bio import motifs

with open(os.path.join(wd, "data/motif", GENE, "meme/meme.xml")) as file:
    meme_record = motifs.parse(file, "MEME")